<a href="https://colab.research.google.com/github/acubillosunal/UBM-MACHINE-LEARNING-1/blob/main/11-homicidios.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


**Recuerda que una vez abierto, Da clic en "Copiar en Drive", de lo contrario no podras alamancenar tu progreso**

Nota: no olvide ir ejecutando las celdas de código de arriba hacia abajo para que no tenga errores de importación de librerías o por falta de definición de variables.

In [ ]:
!wget --no-cache -O init.py -q https://raw.githubusercontent.com/acubillosunal/UBM-MACHINE-LEARNING-1/main/init.py
import init; init.init(force_download=False); 

# Data Lesiones Homicidios

In [ ]:
import pandas as pd
import numpy as np
import os
from datetime import datetime
from unidecode import unidecode

In [ ]:
files_1 = ['./local/data/homicidios/' + str(i) + '.xlsx' for i in range(2010, 2013)]
files_2 = ['./local/data/homicidios/' + str(i) + '.xls' for i in range(2013, 2020)]
files = files_1+files_2
print(files)

In [ ]:
files

In [ ]:
all_dfs = []

for file in files:
    temp = pd.read_excel(file)
    
    # Eliminar acentos en el nombre de las columnas
    temp.columns = [unidecode(str(col)) for col in temp.columns]

    # Reemplazar espacios por _
    temp.columns = temp.columns.str.replace(' ', '_')
    
    # Strip
    temp.columns = temp.columns.str.strip()

    # Convierte todas las letras en minúsculas
    temp.columns = temp.columns.str.lower()

    # Reemplazar NaN, None
    temp.replace('-', np.nan, inplace=True)

    # Set axis name
    temp.rename_axis('id', inplace=True)
    
    all_dfs.append(temp)

In [ ]:
all_dfs

In [ ]:
renaming={
    'clase_sitio':'clase_de_sitio',
    'pais_nace':'pais_de_nacimiento',
    'clase_empleado':'clase_de_empleado'
}

for i in range(len(all_dfs)):
    all_dfs[i].rename(columns=renaming, inplace=True)
    
df_f = pd.concat(all_dfs, axis=0)

In [ ]:
df_f # also try: .describe

In [ ]:
df_f.isnull().sum()

In [ ]:
df_f.drop(columns=['2010','2011','2012','profesion'], inplace=True)

In [ ]:
df_f

In [ ]:
df_f = df_f.reset_index()

In [ ]:
df_f

In [ ]:
df_f.drop(columns=['id'], inplace=True)

In [ ]:
df_f

In [ ]:
df_f = df_f[df_f.isnull().sum(axis=1) < 5]

In [ ]:
df_f

In [ ]:
df_f.isnull().sum()

In [ ]:
df_f.hora = pd.to_datetime(df_f.hora, format='%H:%M:%S')
df_f.fecha = pd.to_datetime(df_f.fecha, format = '%Y- %m - %d %H:%M:%S')
df_f['year'] = df_f.fecha.dt.year
df_f['hora_int'] = df_f.hora.dt.hour

In [ ]:
df_f.edad.replace('NO REPORTADO', np.nan, inplace=True)

In [ ]:
df_f.edad = df_f.edad.astype(float)

In [ ]:
df_f.municipio.replace(' \(CT\)', '', regex=True, inplace=True)
df_f.municipio.replace('BOGOTÁ', 'BOGOTA D.C', inplace=True)
df_f.departamento.replace('BOGOTÁ', 'BOGOTA D.C', inplace=True)
df_f.departamento.replace('GUAJIRA', 'LA GUAJIRA', inplace=True)
df_f.departamento.replace('VALLE', 'VALLE DEL CAUCA', inplace=True)
df_f.departamento.replace('NARIÑO', 'NARINO', inplace=True)

In [ ]:
for col in df_f.columns:
    print(col)
    print(df_f[col].unique())

# EDA

In [ ]:
df_f.columns

In [ ]:
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt

mpl.rcParams['axes.labelsize'] = 14
mpl.rcParams['xtick.labelsize'] = 14
# mpl.rcParams.keys()
sns.set_palette('Accent')

In [ ]:
plt.figure(figsize=(16,9))
ax = sns.lineplot(
                    data=df_f.groupby('fecha').apply(lambda df: df.sexo.value_counts()).reset_index(),
                    x='fecha',
                    y='sexo',
                    hue='level_1'
                 )
 
ax.set_title('Time Series Total Cases Per Day');
ax.legend(['Male', 'Female', 'Not Found']);

In [ ]:
sns.set_style('whitegrid')
plt.figure(figsize=(16,9))
ax = sns.countplot(# count plot can be thought of as a histogram across a categorical, instead of quantitative, variable.
                   data = df_f,
                   x='year',
                   hue='sexo'
                  )

ax.set_title('Counts per Year');
ax.legend(['Mascullino', 'Femenino']);

plt.savefig('figura1.png')

In [ ]:
plt.figure(figsize=(16,9))
ax = sns.countplot(
                    data=df_f,
                    x='departamento'
)

plt.xticks(rotation=45,ha='right');

In [ ]:
plt.figure(figsize=(16,9))

order = ['Lunes', 'Martes', 'Miércoles', 'Jueves', 'Viernes', 'Sábado', 'Domingo']
ax = sns.countplot(
                   data=df_f,
                   x='dia',
                   order=order
                  )

In [ ]:
plt.figure(figsize=(16,9))
ax = sns.countplot(
    data=df_f,
    x='zona',
)

In [ ]:
plt.figure(figsize=(16,9))

sns.barplot(
    data=df_f.clase_de_sitio.value_counts().head(10).to_frame(name='count').reset_index(),
    x='index',
    y='count'
) 

plt.xticks(rotation=45, ha='right');

In [ ]:
plt.figure(figsize=(16,9))

sns.countplot(
    data=df_f,
    x='escolaridad',
);

In [ ]:
plt.figure(figsize=(16,9))

sns.countplot(
    data=df_f,
    x='arma_empleada',
);
plt.xticks(rotation=45, ha='right');

In [ ]:
plt.figure(figsize=(16,9))

sns.barplot(
    data=df_f.profesiones.value_counts().to_frame('count').reset_index().head(10),
    x='index',
    y='count'
) 

plt.xticks(rotation=45, ha='right');

In [ ]:
df_f['hora']

In [ ]:
plt.figure(figsize=(16,9))
sns.countplot(
              df_f.hora_int
             );

plt.savefig('figura_2.png')

In [ ]:
plt.figure(figsize=(16,9))

sns.distplot(
    df_f.edad
);

In [ ]:
sns.set_palette('Set1')
fig, ax = plt.subplots(figsize=(16,9))
sns.violinplot(
    data=df_f[df_f.sexo != "NO REPORTA"],
    x='year',
    y='hora_int',
    hue='sexo',
    ax=ax,
    split=True,
    palette="Accent"
);

In [ ]:
fig, ax = plt.subplots(5, 1, figsize=(16,20), sharex=True)
i=0
for year in np.arange(2015,2020):
    sns.distplot(
        df_f.loc[((df_f.sexo == 'MASCULINO') & (df_f.year == year)), 'hora_int'],
        bins=20,
        color='limegreen',
        label='Male',
        ax=ax[i]
    )

    sns.distplot(
        df_f.loc[((df_f.sexo == 'FEMENINO') & (df_f.year == year)), 'hora_int'],
        bins=20,
        color='orchid',
        label='Female',
        ax=ax[i]
    )
    ax[i].set_title(str(year))
    ax[i].set_xlabel('')
    i+=1

ax[0].legend()
ax[-1].set_xlabel('Hora Del Incidente');

## Maps of counts

In [ ]:
import json

with open('./local/data/geojson_departamentos.json', 'r') as f:
    departamentos = json.load(f)
    
areas = pd.read_json('./local/data/areas_departamentos.json',
                     orient='index')
# with open('./data/geoJson_departamentos.json', 'w') as f:
#     json.dump(departamentos, f)

# with open('./data/areas_departamentos.json', 'w') as f:
#     json.dump(areas, f)

In [ ]:
import plotly.express as px

In [ ]:
df_f.departamento = df_f.departamento.apply(unidecode)

In [ ]:
count_2019 = df_f[df_f.year == 2019].departamento.value_counts().reset_index(name='count')
fig = px.choropleth_mapbox(
    count_2019,
    geojson=departamentos,
    locations='index',
    color='count',
    color_continuous_scale="Viridis",
    mapbox_style="carto-positron",
    zoom=4,
    center = {"lat": 3.47, "lon": -73.96},
    opacity=0.5
)

fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()